In [1]:
import pandas as pd
import numpy as np

In [2]:
from datasets import load_dataset

ds = load_dataset("FiscalNote/billsum")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/91.8M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

data/ca_test-00000-of-00001.parquet:   0%|          | 0.00/6.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [3]:

df = pd.DataFrame(ds['train'])

In [4]:
sample_df = df.sample(n=20, random_state=42).reset_index(drop=True)
sample_df

,text,summary,title
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,Medicare Prescription Drug Integrity Act of 20...,Medicare Prescription Drug Integrity Act of 2013
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,Back to Work Tax Credit - Amends the Internal ...,A bill to amend the Internal Revenue Code of 1...
2,SECTION 1. UTILIZING EVIDENCE FROM CLINICAL EX...,"This bill amends the Federal Food, Drug, and C...","To amend the Federal Food, Drug, and Cosmetic ..."
3,SECTION 1. SHORT TITLE.\n\n This Act may be...,Local Zoning and Property Rights Protection Ac...,Local Zoning and Property Rights Protection Ac...
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,Paterson Great Falls National Historical Park ...,A bill to establish the Paterson Great Falls N...
5,SECTION 1. HONESTY IN SWEEPSTAKES ACT OF 1999....,Honesty in Sweepstakes Act of 1999 - Amends Fe...,Honesty in Sweepstakes Act of 1999
6,SECTION 1. SHORT TITLE.\n\n This Act may be...,"Recognize, Assist, Include, Support, and Engag...","Recognize, Assist, Include, Support, and Engag..."
7,SECTION 1. SHORT TITLE; AMENDMENT OF FEDERAL W...,Clean Water Infrastructure and Security Improv...,To amend the Federal Water Pollution Control A...
8,SECTION 1. SHORT TITLE.\n\n This Act may be...,Leveraging Performance-Based Transportation Se...,Leveraging Performance-Based Transportation Se...
9,SECTION 1. SHORT TITLE.\n\n This Act may be...,Affordable Housing Improvements Act - Amends t...,To authorize the Secretary of Housing and Urba...


In [5]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=d7977abb81a4ac96dce11f6ae24768d7382dee036600731b5cb9ab73f4625e2d
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [6]:
from transformers import AutoTokenizer, AutoModel
from rouge_score import rouge_scorer
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer





[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [17]:
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

def sbert_extractive_summary(text, target_words=200, tolerance=50):
    sentences = sent_tokenize(text)

    # Get sentence embeddings
    embeddings = sbert_model.encode(sentences)

    # Calculate document centroid
    doc_embedding = np.mean(embeddings, axis=0)

    # Score sentences by similarity to centroid
    scores = [(i, np.dot(embeddings[i], doc_embedding))
              for i in range(len(sentences))]

    # Sort by score (descending)
    scores.sort(key=lambda x: x[1], reverse=True)

    # Greedily select sentences until we reach target
    selected_indices = []
    current_words = 0
    min_words = target_words - tolerance
    max_words = target_words + tolerance

    for idx, score in scores:
        sentence_words = len(sentences[idx].split())

        # If adding this sentence keeps us in range, add it
        if current_words + sentence_words <= max_words:
            selected_indices.append(idx)
            current_words += sentence_words

            # Stop if we've reached acceptable range
            if current_words >= min_words:
                break

    # Sort indices to maintain original document order
    selected_indices.sort()

    summary = ' '.join([sentences[i] for i in selected_indices])
    return summary


summaries = []
word_counts = []

for text in sample_df['text']:
    summary = sbert_extractive_summary(text, target_words=200, tolerance=50)
    summaries.append(summary)
    word_counts.append(len(summary.split()))

sample_df['sbert_summary'] = summaries



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
print("\nSample summaries:")
display(sample_df[['text', 'sbert_summary']].head())

from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

rouge_scores = []
for i, row in sample_df.iterrows():
    scores = scorer.score(row['summary'], row['sbert_summary'])
    rouge_scores.append({
        'rouge1': scores['rouge1'].fmeasure,
        'rouge2': scores['rouge2'].fmeasure,
        'rougeL': scores['rougeL'].fmeasure
    })

rouge_df = pd.DataFrame(rouge_scores)
print("\nAverage ROUGE Scores:")
print(rouge_df.mean())


Sample summaries:


,text,sbert_summary
0,SECTION 1. SHORT TITLE.\n\n This Act may be...,1395w-102(e)(3)) is amended--\n ...
1,SECTION 1. SHORT TITLE.\n\n This Act may be...,(c) Simplified Certification.--Section 51(d) o...
2,SECTION 1. UTILIZING EVIDENCE FROM CLINICAL EX...,``(a) In General.--The Secretary shall establi...
3,SECTION 1. SHORT TITLE.\n\n This Act may be...,(b) Notice.--Not later than 30 days after the ...
4,SECTION 1. SHORT TITLE.\n\n This Act may be...,(2) Conditions for establishment.--The Park sh...



Average ROUGE Scores:
rouge1    0.456110
rouge2    0.225021
rougeL    0.291691
dtype: float64


In [19]:
sample_df['sbert_summary'][1]

"(c) Simplified Certification.--Section 51(d) of the Internal \nRevenue Code of 1986, as amended by subsection (b)(2), is amended by \nadding at the end the following new paragraph:\n            ``(16) Simplified certification for qualified long-term \n        unemployed individuals.--\n                    ``(A) In general.--Any individual under paragraph \n                (11) will be treated as certified by the designated \n                local agency as having aggregate periods of \n                unemployment described in such paragraph if the \n                individual is certified by the designated local agency \n                as being in receipt of unemployment compensation under \n                State or Federal law for not less than 6 months during \n                the 1-year period ending on the hiring date. (d) Credit Made Available to Tax-Exempt Employers in Certain \nCircumstances.--Section 52(c) of the Internal Revenue Code of 1986 is \namended--\n            (1) 

In [20]:
sample_df['sbert_summary'][2]

'``(a) In General.--The Secretary shall establish a program to \nevaluate the potential use of evidence from clinical experience--\n            ``(1) to help support the approval of a new indication for \n        a drug approved under section 505(b); and\n            ``(2) to help support or satisfy post-approval study \n        requirements. ``(e) Guidance for Industry.--The Secretary shall--\n            ``(1) utilize the program established in subsection (d), \n        its activities, and any subsequent pilots or written reports, \n        to inform a guidance for industry on--\n                    ``(A) the circumstances under which sponsors of \n                drugs and the Secretary may rely on evidence from \n                clinical experience for the purposes described in \n                subsections (a)(1) or (a)(2); and\n                    ``(B) the appropriate standards and methodologies \n                for collection and analysis of evidence from clinical \n          